In [1]:
import rospy
import time
import math
import jupyros as jr
import numpy as np
import following_goal.msg
import actionlib
import actionlib.msg
import matplotlib.pyplot as plt
import tf
import ipywidgets as widgets

from geometry_msgs.msg import Pose, Point
from following_goal.srv import NumberGoals
from following_goal.msg import PlanningAction
from nav_msgs.msg import Odometry
from IPython.display import display
from sensor_msgs.msg import LaserScan
from matplotlib.animation import FuncAnimation
from nav_msgs.msg import Odometry
from tf.transformations import quaternion_matrix
from std_srvs.srv import *

%matplotlib widget

In [2]:
global user, status

# start the node
rospy.init_node('user_interface')

# initialize the action client
user=actionlib.SimpleActionClient('/reaching_goal', following_goal.msg.PlanningAction)

# wait for the server
user.wait_for_server()

# get the actual state
status=user.get_state()

In [3]:
# Text box for insert the coordinates the goal
goal_x_widget = widgets.FloatText( description='x-coordinate of the goal:', disabled=False)
goal_y_widget = widgets.FloatText( description='y-coordinate of the goal:', disabled=False)

# Confimation button for sending the coordinates of the goal
confirm_goal = widgets.Button(description="Confirm goal")
output_confirm = widgets.Output()

# Confimation button for deleting the goal set
cancell_goal = widgets.Button(description="Cancel the goal")
output_cancell = widgets.Output()

# Box for the menu to set or cancel the goal
menu = widgets.VBox([goal_x_widget, goal_y_widget, confirm_goal, cancell_goal])


In [4]:
global user, status

# Manage the click on the confimation button
def on_confirm_clicked(confirm_goal):
    global x_goal, y_goal
    x_goal = float(goal_x_widget.value)
    y_goal = float(goal_y_widget.value)
    with output_confirm:
        send_goal()
confirm_goal.on_click(on_confirm_clicked)

# Send the coordinated of the goal set
def send_goal():
    goal = following_goal.msg.PlanningGoal()
    status=user.get_state()
    if not (status==actionlib.GoalStatus.ACTIVE):
        goal.target_pose.pose.position.x = x_goal
        goal.target_pose.pose.position.y = y_goal
        user.send_goal(goal)
        print("Goal sent!")
    else:
        print("Goal has been already set!")
        print("Cancel the goal and then set a new one")

# Manage the click on the cancel button
def on_cancell_clicked(cancell_goal):
    with output_cancell:
        cancel_goal()
cancell_goal.on_click(on_cancell_clicked)

# Cancel the goal set
def cancel_goal():
    status = user.get_state()
    if status == actionlib.GoalStatus.ACTIVE:
        user.cancel_goal()
        print("Goal deleted")
    else:
        print("No goal has been set")

# Show the menu for sending or deleting the goal
print("Click on one of the following options:")
display(menu, output_confirm, output_cancell)

Click on one of the following options:


Output()

Output()

In [5]:
global x_goal, y_goal, user, status

class Visualiser:
    def __init__(self):
        # create two subplot, one for tajector and one for reached/deleted goals
        self.fig, (self.ax, self.ax2) = plt.subplots(1,2, figsize=(10,4))
        
        # plot for the tajectory
        self.ln, = self.ax.plot([], [], 'mo', markersize='3', label='Trajectory')
        # plonts for the reached, current and deleted goals
        self.ax.plot(x_goal, y_goal, marker='*', color='green', markersize='8', label='Reached goals')
        self.ax.plot(x_goal, y_goal, marker='*', color='orange', markersize='8', label='Current goal')
        self.ax.plot(x_goal, y_goal, marker='*', color='blue', markersize='8', label='Deleted goals')
        
        #self.x_goal = x_goal
        #self.y_goal = y_goal
        self.x_data, self.y_data = [] , []
        
        # initialize counter for reached and deteleted goals
        self.reached = 0
        self.deleted = 0
        
        # initialize variable for checking the status
        self.check_reached=False
        self.check_deleted=False
    
    def plot_init(self):
        # initialize the limits of the two plots
        self.ax.set_xlim(-30, 30)
        self.ax.set_ylim(-30, 30)
        self.ax2.set_ylim(0, 10)
        
        return self.ln
    
    def odom_callback(self, msg):
        # callback for the current position of the robot
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
        current_px_widget.value=round(msg.pose.pose.position.x, 3)
        current_py_widget.value=round(msg.pose.pose.position.y, 3)
    
    def update_plot(self, frame):
        # updates the two plots
        self.ln.set_data(self.x_data, self.y_data)
        #self.posx=posx
        #self.posy=posy
        #get the status of the robot
        status=user.get_state()
        
        # check if the goal is reached, show * in green
        if status==3 and not(self.check_reached):
            self.ax.plot(x_goal, y_goal, marker='*', color='green', markersize='8', label='Reached')
            self.reached += 1
            self.ax2.bar(['reached', 'deleted'], [self.reached, self.deleted], color = 'b')
            self.check_reached=True
            
        # check if the goal has been deleted, show * in blue
        elif status==2 and not(self.check_deleted):
            self.ax.plot(x_goal, y_goal, marker='*', color='blue', markersize='8', label='Deleted')
            self.deleted += 1
            self.ax2.bar(['reached', 'deleted'], [self.reached, self.deleted], color = 'b')
            self.check_deleted=True
        
        # change the check of reached goal to false
        elif (not(status==3) and self.check_reached):
            self.check_reached=False 
            
        # change the check of deleted goal to false
        elif (not(status==2) and self.check_deleted):
            self.check_deleted=False
            
        # check if the robot is going to the current goal, show * in orange
        elif (not(status==3) and not(status==2)):
            self.ax.plot(x_goal, y_goal, marker='*', color='orange', markersize='8', label='Current')
        
        # show the bar graph to show the number of reached and deleted goals
        self.ax2.bar(['reached', 'deleted'], [self.reached, self.deleted], color = 'm')
        return self.ln

In [6]:
# text box to show the distance from the closest obstacle
min_dist_widget =widgets.FloatText( description='Distance from closest obstacle:', disabled=True)
output_dist=widgets.Output()

# text box to show the x-coordinate of the robot
current_px_widget=widgets.FloatText(description='Robot x-pos:', disabled=True)

# text box to show the y-coordinate of the robot
current_py_widget=widgets.FloatText(description='Robot y-pos:', disabled=True)

In [7]:
# graph for showing trajectory of the robots and goals coordinates
vis = Visualiser()
vis.ax.set_title('Plot of robot trajectory')
vis.ax.set_xlabel('Position on x')
vis.ax.set_ylabel('Position on y')
vis.ax.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.tight_layout()
vis.ax.grid()

# graph for showing the number of reached and deleted goals
vis.ax2.set_title('Number of goals reached and deleted')
vis.ax2.set_xlabel('Goals')
vis.ax2.set_ylabel('Number of goal')

# callback for updating the two graphs
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
time.sleep(0.2)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)

# show graphs
plt.show(block=True)

# show widgets for distance from the obstacle, and current position of the robot
display(min_dist_widget, output_dist)
display(current_px_widget)
display(current_py_widget)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

FloatText(value=0.0, description='Distance from closest obstacle:', disabled=True)

Output()

FloatText(value=6.307, description='Robot x-pos:', disabled=True)

FloatText(value=0.93, description='Robot y-pos:', disabled=True)

In [ ]:
# get the distance from the closest obstacle detected by the laser scanner
global min_range

# find the minor value detected
def scan_callback(scan):
    min_range = scan.range_max
    
    for range_val in scan.ranges:
        if range_val < min_range:
            min_range = range_val
            
    min_dist_widget.value = round(min_range, 3)

jr.subscribe('/scan', LaserScan, scan_callback)
time.sleep(0.3)